# Collaborative Filtering with SVD
Author: Shiyi Wang

In [25]:
import pandas as pd
import numpy as np
from collections import Counter
import random
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix
import pickle
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate,train_test_split,KFold
from surprise import SVD,SVDpp, NMF,SlopeOne,CoClustering
from surprise import accuracy
from collections import defaultdict

In [26]:
df = pd.read_pickle('../data/processed_data.pkl')
df

,user_id,recipe_id,rating,name
0,38094,40893,4,white bean green chile pepper soup
1,1293707,40893,5,white bean green chile pepper soup
3,126440,85009,5,baked potato toppings
4,57222,85009,5,baked potato toppings
5,52282,120345,4,sugared raspberries
...,...,...,...,...
1132360,2002357020,82303,5,easy microwave hot fudge topping
1132361,102526,54493,0,garlic clove chicken
1132364,157126,78003,5,pot roast with port stove top
1132365,53932,78003,4,pot roast with port stove top


Load data using surprise framework

In [35]:
df_copy = df.drop(['name'], axis=1)
df_copy

,user_id,recipe_id,rating
0,38094,40893,4
1,1293707,40893,5
3,126440,85009,5
4,57222,85009,5
5,52282,120345,4
...,...,...,...
1132360,2002357020,82303,5
1132361,102526,54493,0
1132364,157126,78003,5
1132365,53932,78003,4


In [28]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df_copy[['user_id', 'recipe_id', 'rating']], reader)

Use SVD model

In [29]:
algo = SVD(verbose=True)

In [30]:
uid = 226571
iids = [40893, 85009, 82303, 78003]
recipes_names = dict([(rep_id, name)
                     for name, rep_id in zip(df.name, df.recipe_id)])


In [31]:
def refactorRecipeNames(text):
    text = text.replace(" s ", "\'s ")
    text_split = text.split(" ")
    text_split = [t.strip().capitalize() for t in text_split if t != '']
    return " ".join(text_split)


In [32]:
[refactorRecipeNames(recipes_names[r]) for r in iids]

['White Bean Green Chile Pepper Soup',
 'Baked Potato Toppings',
 'Easy Microwave Hot Fudge Topping',
 'Pot Roast With Port Stove Top']

In [33]:
def get_n_predictions(iids, algo, n=10, uid=226571, item_name=recipes_names):

    # create the list to search in
    iid_to_test = [iid for iid in range(231637) if iid not in iids]
    # build data for surprise
    test_set = [[uid, iid, 4.] for iid in iid_to_test]
    # predict
    predictions = algo.test(test_set)
    # get prediction
    pred_ratings = [pred.est for pred in predictions]
    # return top_n indexes
    top_n = np.argpartition(pred_ratings, 1)[-n:]
    # return list of recipe names
    results = [item_name[k] for k in top_n]

    return [refactorRecipeNames(r) for r in results]


In [34]:
get_n_predictions(iids, algo)

AttributeError: 'SVD' object has no attribute 'trainset'